In [69]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [70]:
# Load data
train1 = pd.read_csv("jigsaw-toxic-comment-train.csv")
valid1 = pd.read_csv('validation.csv')
test = pd.read_csv('test.csv')

In [74]:
dataset = pd.read_csv("jigsaw-toxic-comment-train.csv")

In [71]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train = pd.DataFrame()

In [73]:
train1

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['lang']=le.fit_transform(train['lang'])
valid

In [27]:
class_counts = train1[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print(class_counts)

toxic            21384
severe_toxic      1962
obscene          12140
threat             689
insult           11304
identity_hate     2117
dtype: int64


In [44]:
valid

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0
...,...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,it,0
7996,7996,El imbesil ete dela luna no se entera ni ostia...,es,1
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,tr,1
7998,7998,El mapa del reinado de Alhaken esta ligerament...,es,0


In [46]:
valid = pd.DataFrame(valid1.loc[0:100,:])

In [28]:
# Sample 1000 rows from each class
for cls in classes:
    # Sample 1000 rows where the class is 1 (present)
    sampled_class = train1[train1[cls] == 1].sample(n=100, random_state=42)  # Set random_state for reproducibility
    train = pd.concat([train, sampled_class])

train.reset_index(drop=True, inplace=True)


In [48]:
train.shape

(600, 8)

In [49]:
valid.shape

(101, 4)

In [7]:
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [8]:
# Tokenization function using Hugging Face tokenizer
def encode_texts(texts, tokenizer, maxlen=512):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=maxlen,
        return_tensors='pt'
    )


In [50]:
# Encode datasets
x_train = encode_texts(train.comment_text.astype(str), tokenizer, maxlen=192)
x_valid = encode_texts(valid.comment_text.astype(str), tokenizer, maxlen=192)
x_test = encode_texts(test.content.astype(str), tokenizer, maxlen=192)

In [62]:
y_train = torch.tensor(train.toxic.values)
y_valid = torch.tensor(valid.toxic.values)

In [52]:
# Custom dataset class
class ToxicCommentsDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [53]:
# Dataset and Dataloader
train_dataset = ToxicCommentsDataset(x_train, y_train)
valid_dataset = ToxicCommentsDataset(x_valid, y_valid)

train_loader = DataLoader(train_dataset, batch_size=16)  # Try reducing batch size
valid_loader = DataLoader(valid_dataset, batch_size=16)


In [54]:
import torch
print(torch.version.cuda)  # This should print a CUDA version number if supported
print(torch.cuda.is_available())  # This should print True if CUDA is available

None
False


In [55]:
# Load pre-trained BERT model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=1
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cpu


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [56]:
# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
total_steps = len(train_loader) * 3  
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)

# Loss function
loss_fn = torch.nn.BCEWithLogitsLoss()

d:\ML\Intel\.conda\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [57]:
# Training loop
def train_epoch(model, dataloader, optimizer, device, scheduler):
    model.train()
    total_loss = 0
    correct_predictions = 0
    
    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].unsqueeze(1).float().to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [58]:
# Validation loop
def eval_model(model, dataloader, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].unsqueeze(1).float().to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [59]:
# Training
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    val_loss = eval_model(model, valid_loader, device)
    print(f"Train Loss: {train_loss}, Validation Loss: {val_loss}")

Epoch 1/3


100%|██████████| 38/38 [02:08<00:00,  3.38s/it]


Train Loss: 0.4274276894958396, Validation Loss: 1.334118332181658
Epoch 2/3


100%|██████████| 38/38 [02:08<00:00,  3.38s/it]


Train Loss: 0.17943686442939857, Validation Loss: 1.7722059999193465
Epoch 3/3


100%|██████████| 38/38 [02:09<00:00,  3.40s/it]


Train Loss: 0.1587729261893975, Validation Loss: 1.8785559960774012


In [60]:
test.shape

(63812, 3)

In [61]:
test1 = pd.DataFrame(test.loc[0:200,:])

In [63]:
x_test = encode_texts(test1.content.astype(str), tokenizer, maxlen=192)

In [67]:
x_train

{'input_ids': tensor([[   101,  10198,  11263,  ...,      0,      0,      0],
        [   101,  32470,    117,  ...,      0,      0,      0],
        [   101,    107, 101270,  ...,  14962,  10944,    102],
        ...,
        [   101,  10377,  10124,  ...,      0,      0,      0],
        [   101,  13997,  12377,  ...,      0,      0,      0],
        [   101,  12271,  11263,  ...,      0,      0,      0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [68]:
# Test Dataset
test_dataset = ToxicCommentsDataset(x_test)

test_loader = DataLoader(test_dataset, batch_size=32)

# Prediction
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.sigmoid(outputs.logits).cpu().numpy()
        predictions.extend(preds)

# Convert predictions to a binary format if needed
binary_predictions = [1 if pred >= 0.5 else 0 for pred in predictions]

TypeError: object of type 'NoneType' has no len()